In [1]:
import os
os.environ["COHERE_API_KEY"] = 'E2efronXbGqXSR0sclyInSjXptFOjdIjsbCeXFwP'

import pandas as pd
from tqdm.notebook import tqdm

from langchain_cohere import ChatCohere
from langchain_ollama import ChatOllama

from langchain_core.prompts import ChatPromptTemplate
from langchain_pymupdf4llm import PyMuPDF4LLMLoader

from pydantic import BaseModel, Field

In [2]:
XL_PATH = r"../dataset/screening/articles_to_screen.csv"
OUT_DIR = r"results"
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

In [3]:
# screened abstracts

df = pd.read_csv(XL_PATH)[["key", "title", "abstract"]]

display(df.head())
print(len(df))

,key,title,abstract
0,rayyan-97865221,Responsive Magnetic Particle Imaging Tracer: O...,Magnetic particle imaging (MPI) has emerged as...
1,rayyan-97865222,"The role of apoptosis, immunogenic cell death,...","Keloids, characterized by excessive extracellu..."
2,rayyan-97865224,"Particle tracking, recognition and LET evaluat...",Objective. This study aims to assess the compo...
3,rayyan-97865227,Understanding Relative Biological Effectivenes...,Purpose: Recent experimental studies and clini...
4,rayyan-97865228,Use of parvovirus B19-like particles in self-i...,Bioluminescence resonance energy transfer phot...


413


In [4]:
class OutputSchema(BaseModel):
    thoughts: str = Field(description= "thoughts of the model")
    decision: int = Field(description="1 if the article is selected, 0 otherwise")
    reason: str = Field(description = "generate a consise one sentence long reason for the decision")
    keywords: str = Field(description="list all the important keywords associated with the decision")


# llm = ChatCohere(model="command-r", temperature=0.0)
llm = ChatOllama(model="deepseek-r1:32b", temperature=0.0, num_ctx=5_000)
struct_llm = llm.with_structured_output(OutputSchema)

In [ ]:
# messages = [
#     ('system',  "You are a helpful AI agent that assists in accurately screening the article using its abstract. " 
#                 "Based solely on the abstract provided, determine whether the article discusses an APPLICATION OF AI METHODS IN CARBON ION THERAPY? "
#                 "Your decision should be '0' for NO or '1' for YES. Then, generate a concise, one-sentence reason for your decision."
#     ),
#     ('human', "abstract:\n\n title: {title}, \n content: {abstract}")
#   ]

messages = [
    ('system',  "You are a helpful AI assistant that accurately screens and SELECTS ORIGINAL RESEARCH ARTICLES relevant to given literature review TOPIC, based solely on their ABSTRACT." 
                "Your decision should be '1' if SELECTED or '0' otherwise. Generate a concise, one-sentence reason to motivate your decision. Also list the important keywords associated with the decision"
    ),
    ('human', "TOPIC: APPLICATIONS OF AI METHODS IN CARBON ION THERAPY\n\nABSTRACT:\n\n title: {title}, \n content: {abstract}")
  ]

prompt_template = ChatPromptTemplate.from_messages(messages)
chain = prompt_template | struct_llm

decision_df = {"key":[], "title":[], "abstract":[], "decision":[], "ai_methods":[], "reason":[], "thoughts":[]}

rows = [row.to_dict() for i,row in df.iterrows()]

for row in tqdm(rows):
  # key, title, selection = row.to_dict()["key"], row.to_dict()["title"], row.to_dict()["selection"]

  inputs = {key:row[key] for key in ["title", "abstract"]} 

  output = chain.invoke(inputs)

  for key,val in {**row, **dict(output)}.items():
      decision_df[key].append(val)
  
decision_df = pd.DataFrame(decision_df)
if not os.path.exists(OUT_DIR):
   os.makedirs(OUT_DIR)
decision_df.to_csv(os.path.join(OUT_DIR, "ai_decision.csv"), index=False)


  0%|          | 0/413 [00:00<?, ?it/s]

In [17]:
decision_df[decision_df.decision==1]

,key,title,abstract,decision,ai_methods,reason,thoughts
3,rayyan-97865227,Understanding Relative Biological Effectivenes...,Purpose: Recent experimental studies and clini...,1,Modified Microdosimetric Kinetic Model (mMKM),The abstract discusses the application of the ...,"Okay, so I need to determine whether this abst..."
18,rayyan-97865248,A dosiomics approach to treatment outcome mode...,PURPOSE: To investigate the role of dosiomics ...,1,Regularized Cox proportional hazard model (r-C...,The study applies machine learning methods (r-...,"Okay, so I need to determine whether this abst..."
19,rayyan-97865249,Deep learning-based voxel sampling for particl...,Objective.Scanned particle therapy often requi...,1,"Deep learning, P-Net",The study presents a novel deep-learning model...,"Okay, so I need to determine whether this abst..."
21,rayyan-283077592,Deep-learning synthetized 4DCT from 4DMRI of t...,PURPOSE: To investigate the feasibility of dee...,1,Generative Adversarial Network (cGAN),The study presents an original research on usi...,"Okay, so I need to determine whether this abst..."
24,rayyan-283077595,Evaluation of machine learning models for pred...,BACKGROUND AND PURPOSE: Few studies have exami...,1,"Support Vector Machine (SVM), Logistic Regression",Evaluates machine learning approaches for pred...,"Okay, so I need to determine whether this abst..."
25,rayyan-283077596,Normal tissue complication probability model f...,BACKGROUND AND PURPOSE: To develop a normal ti...,1,"radiomics, dosiomics",The study develops an NTCP model using radiomi...,"Okay, so I need to determine whether this abst..."
28,rayyan-283077603,Deep learning-based Monte Carlo dose predictio...,BackgroundOnline adaptive radiotherapy (OART) ...,1,"Deep Learning, U-Net, Channel Attention Mechanism",The study presents a novel deep learning model...,"Okay, so I need to determine whether this abst..."
29,rayyan-283077605,Mixed- and multi-relative biological effective...,Background and purpose: In carbon ion radiothe...,1,Monte Carlo simulation,The study introduces an optimization framework...,"Okay, so I need to determine whether this abst..."
32,rayyan-283077608,Validation of the generalized stochastic micro...,Objective. The present work shows the first ex...,1,Generalized Stochastic Microdosimetric Model (...,The study presents an original model (GSM²) va...,"Okay, so I need to determine whether this abst..."
44,rayyan-283077623,Dosimetric performance of cone beam CT-guided ...,Purpose: We investigated whether an ultra-hypo...,1,"Cone-beam CT (CBCT), Deformable image registra...",The study investigates the use of cone-beam CT...,"Okay, so I need to determine whether this abst..."


In [ ]:
# generating summaries

In [ ]:
# this doesn't work very well
# from langchain_community.document_loaders import PyPDFLoader


# file_path = r"1-s2.0-S1120179724002163-main.pdf"
# loader = PyPDFLoader(file_path, mode='single')
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

# manuscript = "\n".join([page.page_content for page in pages])

In [ ]:
# import pymupdf4llm

# file_path = r"1-s2.0-S1120179724002163-main.pdf"
# manuscript = pymupdf4llm.to_markdown(file_path)

# ref_index = manuscript.lower().find("references")

# # Keep everything up to and including "References"
# if ref_index != -1:
#     manuscript = manuscript[:ref_index + len("references")]

# # Now md_text contains only up to the References section
# print(manuscript)

In [ ]:

# class OutputSchema(BaseModel):
#     thoughts: str = Field(description= "thoughts of the model")
#     study_objective: str = Field(description="The objective of the article")
#     ai_methods: str = Field(description="The AI method(s) used in the article in the context of carbon ion therapy. Describe each AI method at least in one sentence")
#     application_domain: str = Field(description="the specific carbon ion therapy application domain")
#     dataset_characteristics: str = Field(description="characteristics of the dataset, including the training, validation, test split; also external validation information, if present")
#     key_findings: str = Field(description="key findings mentioned in the article")
#     pros_and_cons: str = Field(description="strengths and limitations of the study")
#     future_direction: str = Field(description="future directions (if stated)")

# llm = ChatCohere(model='command-r', temperature=0.0)
# # llm = ChatOllama(model="deepseek-r1:32b", temperature=0.0)
# struct_llm = llm.with_structured_output(OutputSchema)

# messages = [
#     ('system',  "You are a helpful AI agent that assists in accurately summarizing the article in a standardized format "
#                 "to be used for the literature review titled 'APPLICATIONS OF AI IN CARBON ION THERAPY'. \n"
#                 "Summarize the article under the following standard headers:\n"
#                 "'1. Study objective', '2. AI methods used', '3. carbon therapy application domain', '4. Dataset characteristics', '5. Key findings', "
#                 "'6. Strengths and limitations', '7. Future directions (if stated)' "
#     ),
#     ('human', "article: \n {article}")
#   ]

# prompt_template = ChatPromptTemplate.from_messages(messages)
# chain = prompt_template | struct_llm


In [4]:

class OutputSchema(BaseModel):
    thoughts: str = Field(description="thoughts of the model")
    aim: str = Field(description="answer to q1: aim of the research article")
    category: str = Field(description="answer to q2: article category")
    dataset: str = Field(description="answer to q3: dataset description and the strategy associated with training, validation and test involved in AI modelling. Also provide the sample counts associated with the train, val and test set.")
    ai_methodology: str = Field(description="answer to q4: the AI methodology used by the authors for their analysis")
    pros_and_cons: str = Field(description="answer to q5: the strengths and weaknesses of the methodology followed")
    results: str = Field(description="answer to q6: summary of the results in terms of the performance metrics and the appropriateness of the metrics were to evaluate the AI model.")
    arguments: str = Field(description="answer to q7: the strong and weak arguments that the authors are pointing out in the discussion")
    conclusion: str = Field(description="answer to q8. their conclusion, and main arguments to support it")
    critical_summary: str = Field(description="a critical summary combining the answers for q1-q8")
    short_summary: str = Field(description="a concise critical summary limited 300 characters, including spaces")

# llm = ChatCohere(model='command-r', temperature=0.0)
llm = ChatOllama(model="deepseek-r1:32b", temperature=0.0, num_ctx=15_000)
struct_llm = llm.with_structured_output(OutputSchema)

In [ ]:
# from transformers import AutoTokenizer

# import json
# schema_str = json.dumps(OutputSchema.model_json_schema())
# tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-32B")
# len(tokenizer.encode(schema_str))

In [5]:
system_message = ''''
You are an expert AI reviewer tasked to perform a concise yet accurate literature review of the provided ARTICLE for the specified REVIEW_TOPIC.
Carefully read the entire ARTICLE (exluding the abstract) and generate a CRITICAL_SUMMARY and a SHORT_SUMMARY by accurately answering each of the QUESTIONS below.

QUESTIONS:
q1. What is the aim of this study?
q2. Select the most appropriate category the article belong to from the LIST below? 
    LIST:  ['Treatment planning, optimization and verification', 'Synthetic imaging', 'Tumor control probability (TCP) prediction', 'Normal tissue complication probability (NTCP) prediction']
    - Treatment planning, optimization, and verification
        *Focus:* all processes before treatment delivery, including treatment dose prediction, dose planning, plan optimization, dose calculation, and plan verification
    - Synthetic imaging 
        *Focus:* generating/transforming images for treatment planning or dose delivery.
    - Tumour control probability (TCP) prediction 
        *Focus:* modelling tumor response to the treatment
    - Normal tissue complication probability (NTCP) prediction
        *Focus:* adverse events/toxicities/complications/quality-of-life after treatment.
    If the study falls into multiple categories, select the one stated as the **primary endpoint**; if not stated, choose the one most emphasized in the title and/or aim. 
    **Do not include your reasoning or deliberation process in the answer for q2. **
q3. Describe the dataset and provide the training, validation and test strategy involved in AI modelling, including the corresponding sample counts.
q4. Define the AI methodology used by the authors for their analysis.
q5. What are the strengths and weaknesses of the methodology followed?
q6. Can you summarize the results in terms of the performance metrics and assess whether the authors chose appropriate metrics to evaluate the AI model?
q7. What are the strong and weak arguments that the authors are pointing out in the discussion?
q8. What is their conclusion, and what are their main arguments to support it?

Answer each question accurately using the information directly available in the ARTICLE. 
If specific information required to answer any question is not present in the ARTICLE, state this clearly instead of speculating.
The ANSWERS should be detailed, evidence-based and written in an objective and academic tone appropriate for a scientific literature review. 

The CRITICAL_SUMMARY should integrate the answers to q1-q8 into a coherent, structured review.
The SHORT_SUMMARY should be a compressed version of CRITICAL_SUMMARY, limited to 300 characters (including spaces).
'''

human_message = "ARTICLE:\n{article}\n\nREVIEW_TOPIC:{topic}"

In [6]:
messages = [
    ('system', system_message),
    ('human', human_message)
]

prompt_template = ChatPromptTemplate.from_messages(messages)
chain = prompt_template | struct_llm

In [7]:
import re

In [9]:
outputs = []

ARTICLE_DIR = r"selected_articles"

file_paths = [os.path.join(ARTICLE_DIR, file) for file in os.listdir(ARTICLE_DIR) if file.endswith(".pdf")]

for file_path in tqdm(file_paths):
    loader = PyMuPDF4LLMLoader(file_path, mode='single') #we are loading all the pages of the documents in a single page
    docs = loader.load()
    manuscript = docs[0].page_content

    # Keep everything except contents inside "References"
    pattern = re.compile(r'^[#\*\s]*references\b', re.IGNORECASE | re.MULTILINE)
    match = pattern.search(manuscript)
    if match:
        ref_index = match.start()
        manuscript = manuscript[:ref_index + len(match.group(0))]

  

    output = dict(chain.invoke({"article":manuscript, "topic":"APPLICATIONS OF AI METHODS IN CARBON ION THERAPY"}))
    output["file_path"] = file_path

    print(output["category"], file_path, output['aim'])




  0%|          | 0/15 [00:00<?, ?it/s]

Normal tissue complication probability (NTCP) prediction selected_articles/Zhang_Predicting_Weight_Loss_PT_2018_JJCO.pdf The study aims to predict critical weight loss in cancer patients undergoing particle therapy by identifying key predictors and building a prediction model.
Synthetic imaging selected_articles/Parrella_SyntCT_CIRT_Abdomen_Bioeng_2023.pdf The aim is to evaluate if a cGAN can generate synthetic CTs for CIRT in abdominal sites, supporting MRI-only workflows.
Treatment planning, optimization and verification selected_articles/Zhang_DL_Denoising_MC_DoseCalculationCIRT_MP_2023.pdf The study aims to evaluate the feasibility of using deep learning-based Monte Carlo (MC) denoising methods to accelerate plan verification in carbon ion radiotherapy by reducing simulation time while maintaining accuracy.
Treatment planning, optimization and verification selected_articles/Yabe_DoseDistribPrediction_DL_PT_2020_MP.pdf The aim of this study is to predict dose distributions from lumi

In [10]:
output

{'thoughts': "Okay, so I need to help this user by providing a literature review based on the provided article. The topic is applications of AI methods in carbon ion therapy. Let me start by understanding what each part of the query requires. They want a CRITICAL_SUMMARY and a SHORT_SUMMARY, both answering specific questions about the study. The questions are from q1 to q8, covering aim, category, dataset, methodology, strengths/weaknesses, results, arguments, and conclusion. I need to make sure each answer is accurate and based on the article provided. Also, the CRITICAL_SUMMARY should integrate all these answers into a coherent review, while the SHORT_SUMMARY is a concise version limited to 300 characters. I'll go through each question one by one, extract the necessary information from the article, and structure the responses appropriately. It's important to be objective and evidence-based, avoiding any speculation where data isn't available. Let me start with q1: What is the aim of 

In [ ]:
# from fpdf import FPDF


# def clean_text(text):
#     return text.encode('latin-1', errors='replace').decode('latin-1')

# # Create PDF
# pdf = FPDF()
# pdf.add_page()
# pdf.set_font("Arial", size=12)


# for heading, text in output.items():
#     # Add heading
#     pdf.set_font("Arial", 'B', 14)
#     pdf.cell(0, 10, heading, ln=True)
    
#     # Add text
#     pdf.set_font("Arial", size=12)
#     pdf.multi_cell(0, 10, clean_text(text))
#     pdf.ln(5)  # Add some space

# pdf.output(os.path.join("results", "output.pdf"))

In [ ]:
# from transformers import pipeline, AutoConfig
# from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
# from langchain_core.prompts import PromptTemplate

# model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B" #"deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"

# config = AutoConfig.from_pretrained(model_name)
# pipe = pipeline("text-generation", model=model_name, max_new_tokens=2048, device=1, do_sample=False) #temperature = 0.0 since do_sample is False
# llm = HuggingFacePipeline(pipeline=pipe)


# # print(config)
